# Исследование по метрикам на основе данных моего произношения

## Импорт библиотеки

In [97]:
# Импорт библиотеки
import sys
import os
import re
import numpy as np
from jiwer import wer, cer, mer, wil

import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Сбор список слов
from collection import collection

## Коллекций слова

In [3]:
for theme, words in collection.items():
    print(f"Тема: {theme}")
    print("Слова:", ', '.join(words))
    print("\n")

## Подключение к БД

In [4]:
# Соединение с БД
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="dataset_speech_recognition",
    user="postgres",
    password="postgres"
)
cur = conn.cursor()

In [5]:
# Получение данных из таблицы
cur.execute("SELECT * FROM recognition_data")
data = cur.fetchall()

In [6]:
# Закрытие соединения
cur.close()
conn.close()

## Создание и анализ датафрейма

In [7]:
# Создание DataFrame из полученных данных
df = pd.DataFrame(data, columns=['id', 'data_recognition', 'transcription_word', 'word_for_check', 'date_recoding', 'wer', 'cer', 'mer', 'wil', 'iwer'])

In [8]:
# Преобразование столбца date_recoding в формат datetime
df['date_recoding'] = pd.to_datetime(df['date_recoding'])

In [9]:
df.info()

In [10]:
df.head()

In [11]:
# Экспорт DataFrame в CSV файл
df.to_csv('database.csv', index=False)

##### Поскольку модель выдавала текст с знаками и другие, поэтому будем устранять их и заново рассчитать метрику

## Предобработка данных

### Изменение индеса в временный ряд

In [21]:
# Отбросим ненужные столбцы
research_df = df.drop(['id', 'data_recognition'], axis=1)

# Установим индекс по столбцу 'date_recoding'
research_df = research_df.set_index('date_recoding')

# Убедимся, что индекс отсортирован
research_df = research_df.sort_index()

In [22]:
research_df.info()

In [23]:
research_df.head()

### Установка диапазона времени дат

In [25]:
# Фильтруем данные по диапазону дат
start_date = '2024-04-22'
end_date = '2024-05-17'

research_df = research_df.loc[start_date:end_date]

### Удаление лишних знаков в тексте

In [26]:
def clean_text(text):
    # Удаление всех знаков пунктуации
    return re.sub(r'[^\w\s]', '', text)

In [27]:
research_df['transcription_word'] = research_df['transcription_word'].apply(clean_text)

In [28]:
research_df.head()

### Установка метрик

In [29]:
def iwer(reference_sentence, hypothesis_sentence) -> float:
    """
    Вычисляет Inflectional Word Error Rate (IWER) между предложением-эталоном и гипотезой.

    Параметры:
    reference_sentence (str): Предложение-эталон (правильный вариант).
    hypothesis_sentence (str): Гипотеза (предсказанный вариант).

    Возвращает:
    float: Значение Inflectional Word Error Rate (IWER) в процентах.
    """
    reference_words = reference_sentence.split()
    hypothesis_words = hypothesis_sentence.split()

    # Находим количество слов в предложении-эталоне
    total_words = len(reference_words)

    # Считаем количество неправильно распознанных слов
    incorrect_words = sum(1 for ref, hyp in zip(reference_words, hypothesis_words) if ref != hyp)

    # Вычисляем Inflectional Word Error Rate (IWER)
    iwer_score = incorrect_words / total_words

    return iwer_score

In [30]:
def calculate_metrics(df):
    df['wer'] = df.apply(lambda row: wer(row['word_for_check'], row['transcription_word']), axis=1)
    df['cer'] = df.apply(lambda row: cer(row['word_for_check'], row['transcription_word']), axis=1)
    df['mer'] = df.apply(lambda row: mer(row['word_for_check'], row['transcription_word']), axis=1)
    df['wil'] = df.apply(lambda row: wil(row['word_for_check'], row['transcription_word']), axis=1)
    df['iwer'] = df.apply(lambda row: iwer(row['word_for_check'], row['transcription_word']), axis=1)

    return df

In [31]:
# Вычисление метрик
research_df = calculate_metrics(research_df)

In [32]:
research_df.head(10)

In [33]:
research_df[research_df['word_for_check'] == 'робот'].head

## Анализ данных

In [89]:
# Сохраняем метрики
metrics = ['wer', 'cer', 'mer', 'wil', 'iwer']

# Словарь цветов для метрик
colors = {
    'wer': 'blue',
    'cer': 'orange',
    'mer': 'green',
    'wil': 'black',
    'iwer': 'purple'
}

### Функция отрисовки графики

In [90]:
def draw_plot(word):
    # Фильтрация данных по слову
    filtered_df = research_df[research_df['word_for_check'] == word].sort_index()

    for metric in metrics:
        # Построение графика
        plt.figure(figsize=(15, 5))
        plt.plot(filtered_df.index, filtered_df[metric], marker='o', linestyle='-', color=colors[metric], label=metric.upper())
        
        # Скользящее среднее
        rolling_mean = filtered_df[metric].rolling(window=3).mean()
        plt.plot(filtered_df.index, rolling_mean, color='red', linestyle='--', label=f'{metric.upper()} (3-дневное скользящее среднее)')
        
        plt.title(f'Динамика {metric.upper()} для слова "{word}"', fontweight='bold', fontsize=12)
        plt.xlabel('Дата', fontweight='bold', fontsize=12)
        plt.ylabel(metric.upper(), fontweight='bold', fontsize=12)
        plt.grid(True)
        plt.xticks(rotation=45)
        
        # Добавление надписей
        for idx, row in filtered_df.iterrows():
            plt.annotate(row['transcription_word'], (idx, row[metric]), textcoords="offset points", xytext=(0,10), ha='center', rotation=45)
        
        plt.legend()
        plt.tight_layout()
        
        # Показ графика
        plt.show()

### Слово "Робот"

In [91]:
draw_plot('робот')

### Слово "Инженер"

In [60]:
draw_plot('инженер')

In [107]:
# Функция для вычисления изменений метрик
def compute_metrics_change(group):
    metrics = ['wer', 'cer', 'mer', 'wil', 'iwer']
    changes = {metric: (group[metric].iloc[-1] - group[metric].iloc[0]) for metric in metrics}
    
    return changes

In [108]:
# Функция для построения графиков изменения метрик по темам
def draw_change_plot_by_theme(collection):
    metrics = ['wer', 'cer', 'mer', 'wil', 'iwer']

    for theme, words in collection.items():
        # Фильтрация данных по словам из текущей темы
        theme_df = research_df[research_df['word_for_check'].isin(words)]

        if theme_df.empty:
            continue

        # Группировка по слову и вычисление изменений метрик
        grouped_df = theme_df.groupby('word_for_check')
        changes = {word: compute_metrics_change(group) for word, group in grouped_df}

        for metric in metrics:
            # Создание графика для текущей метрики
            plt.figure(figsize=(15, 5))
            words = list(changes.keys())
            values = [changes[word][metric] for word in words]

            # Отфильтруем слова с некорректными значениями
            words = [word for word, value in zip(words, values) if pd.notnull(value) and np.isfinite(value)]
            values = [value for value in values if pd.notnull(value) and np.isfinite(value)]

            # Определение направления изменения
            arrows = ['↓' if val < 0 else '↑' for val in values]
            labels = [f'{val:.2f} {arrow}' for val, arrow in zip(values, arrows)]
            colors = ['green' if val < 0 else 'red' for val in values]

            # Построение столбчатой диаграммы
            plt.bar(words, values, color=colors)

            # Добавление подписей
            for i, (word, label) in enumerate(zip(words, labels)):
                plt.text(i, values[i], label, ha='center', va='bottom' if values[i] < 0 else 'top')

            plt.title(f'Изменение {metric.upper()} по теме "{theme}"', fontweight='bold', fontsize=12)
            plt.xlabel('Слово', fontweight='bold', fontsize=12)
            plt.ylabel(f'Изменение {metric.upper()}', fontweight='bold', fontsize=12)
            plt.grid(True)
            plt.xticks(rotation=45)
            
            plt.tight_layout()
            plt.show()

In [109]:
# Пример вызова функции
draw_change_plot_by_theme(collection)